# Data Aggregation & Summaries

Use this Jupyter Notebook to take our initial dataset(s) and generate aggregate/reformatted/summarized CSVs usable by our frontend.

In [1]:
# imports and initial variables
import pandas as pd
import numpy as np

json_out_dir = "../website/src/lib/data/"
dataset_name = "census_tracts_processed.csv"
dataset_df = pd.read_csv(dataset_name, dtype={"geoid": str})

In [7]:
# create aggregate info for each county
# GEOID consists of 11 digits, first 2 are state and next 3 are county
# matches county ID in map
df = dataset_df.copy()
df["id"] = df["geoid"].astype(str).str[:5].astype(int)

# calculate aggregates
def mean_pos(x):
    pos_vals = x[x > 0]
    return np.round(np.mean(pos_vals), 2)

def listify(x):
    return list(set(x))
    
agg = {
    "median_income": mean_pos,
    "median_home_value": mean_pos,
    "name": listify,
    "city": lambda x: listify(x)[0],
    "metro_area": lambda x: listify(x)[0],
}
agg_non_default = agg.keys()
agg.update({
    col: "sum" for col in df.columns if col not in [*agg_non_default, "id", "geoid"]
})
agg_df = df.groupby("id").agg(agg).reset_index()

# clusters
metro_areas = {item: index for index, item in enumerate(agg_df["metro_area"].unique())}
agg_df["area_cluster"] = agg_df["metro_area"]
agg_df["area_cluster"] = agg_df["area_cluster"].map(metro_areas)

# for tracts in agg_df["name"]:
#     county = None
#     state = None
#     for tract in tracts:
#         s = tract.split(", ")
#         if county is None:
#             county = s[1]
#             state = s[2]
#         else:
#             assert county == s[1] and state == s[2], "diff county detected! " + county + state

# extract data from name (county/state is clean, ie only one exists per name)
agg_df["name"] = agg_df["name"].str[0]
agg_df["tract_count"] = len(agg_df["name"])
agg_df[["name", "county", "state"]] = agg_df["name"].str.split(", ", expand=True)
agg_df = agg_df.drop(columns=["name"])

# recalculate proportion 25 under
agg_df["proportion_25_under"] = agg_df["total_population_25_under"] / agg_df["total_population"]

# reorder columns
agg_df.insert(5, "tract_count", agg_df.pop("tract_count"))
agg_df.insert(5, "state", agg_df.pop("state"))
agg_df.insert(3, "county", agg_df.pop("county"))
agg_df.insert(6, "area_cluster", agg_df.pop("area_cluster"))

# convert whole number fields to ints
int_cols = ["total_population",
       "total_population_25_over", "educational_attainment", "white_alone",
       "black_alone", "native_alone", "asian_alone",
       "native_hawaiian_pacific_islander", "some_other_race_alone",
       "two_or_more", "hispanic_or_latino", "total_population_25_under"]
agg_df[int_cols] = agg_df[int_cols].astype(int)

# save df
agg_df.to_csv("county_aggregated.csv", index=False)

# get json
agg_df.to_json(json_out_dir + "county_aggregated.json", orient="records")

In [ ]:
# avg national housing cost increase
# see https://fred.stlouisfed.org/series/ASPUS

adj_df = pd.read_csv("county_aggregated.csv")
prices = pd.read_csv("avg_housing_increase_2000_2024_oct.csv")

print(prices)
avg_increase = int(prices.iloc[len(prices) - 1]["ASPUS"]) - int(prices.iloc[0]["ASPUS"])
print(avg_increase)

   observation_date   ASPUS
0        2000-01-01  202900
1        2000-04-01  202400
2        2000-07-01  204100
3        2000-10-01  212100
4        2001-01-01  211000
..              ...     ...
95       2023-10-01  498300
96       2024-01-01  519700
97       2024-04-01  502200
98       2024-07-01  498700
99       2024-10-01  510300

[100 rows x 2 columns]
307400


In [6]:
# Home frequently associated words

from collections import Counter

ignore = ["", "a", "and", "to", "the", "is", "Habitat", "for", "in", "\n\n—", "home", "place", "of", "we", "our", "as", "with", "her", "how", "where", "A", "new", "Humanity", "on", "their", "you", "that", "are", "live", "Home", "can", "“Home", "I", "my", "does", "—", "build", "will", "share", "they"]

with open("habitat-page-stripped.txt", "r") as file:
    file_content = file.read()
counts = Counter([word for word in file_content.split(" ") if word not in ignore])
print(counts.most_common(20))


[('family', 5), ('help', 3), ('think', 3), ('means', 3), ('had', 3), ('could', 3), ('were', 3), ('from', 3), ('base', 3), ('each', 3), ('or', 3), ('was', 3), ('one', 3), ('by', 3), ('Donate', 3), ('she', 2), ('mother', 2), ('mean', 2), ('mom', 2), ('way', 2)]
